In [43]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [44]:
data = pd.read_csv("../heart.csv")[:300]
features = shuffle(data)
features.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
169,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
118,46,0,1,105,204,0,1,172,0,0.0,2,0,2,1
117,56,1,3,120,193,0,0,162,0,1.9,1,0,3,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
86,68,1,2,118,277,0,1,151,0,1.0,2,1,3,1


In [45]:
labels = features.pop("output")

In [46]:
features = np.array(features)
labels = np.array(labels).reshape(-1,1)

In [47]:
features = ss().fit_transform(features)
features[0]

array([-0.14361012,  0.68074565, -0.94274025,  0.47815504, -0.85203011,
        2.41209076, -1.00257359,  0.2315564 ,  1.43569652,  1.78764939,
       -2.27454616, -0.70698293,  1.12842152])

In [48]:
from torch.utils.data import DataLoader, TensorDataset

features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

In [49]:
print(features.shape, labels.shape)
print(features[0])

torch.Size([300, 13]) torch.Size([300, 1])
tensor([-0.1436,  0.6807, -0.9427,  0.4782, -0.8520,  2.4121, -1.0026,  0.2316,
         1.4357,  1.7876, -2.2745, -0.7070,  1.1284])


In [50]:
dataset = TensorDataset(features, labels)

In [51]:
BATCH_SIZE = 30
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)

print(len(dataloader))

10


In [52]:
model = torch.nn.Sequential(
    torch.nn.Linear(13,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,5),
    torch.nn.ReLU(),
    torch.nn.Linear(5,1),
    torch.nn.Sigmoid()
)

In [53]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [54]:
epochs = 1000

for epoch in range(epochs):
    for features,labels in dataloader:
        preds = model(features)
        loss = loss_fn(preds, labels)

        loss.backward()

        opt.step()
        opt.zero_grad()


    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for features,labels in dataloader:
                preds = model(features)
                for i in range(len(preds)):
                    if preds[i][0] >= 0.5:
                        preds[i][0] = 1
                    else:
                        preds[i][0] = 0

                    if preds[i][0] == labels[i][0]:
                        right += 1

            

            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ (len(dataloader)*BATCH_SIZE), 2)}%")


Loss : 0.17536266148090363
Accuracy : 89.33%
Loss : 0.09613879024982452
Accuracy : 91.67%
Loss : 0.1040271669626236
Accuracy : 93.0%
Loss : 0.25675690174102783
Accuracy : 95.0%
Loss : 0.1981516033411026
Accuracy : 96.0%
Loss : 0.0996943935751915
Accuracy : 97.0%
Loss : 0.08306200057268143
Accuracy : 97.33%


KeyboardInterrupt: 